## Manual publication DB insertion from raw text using syntax features
### Publications and conferences of Dr. AVRAM Sanda, Profesor Universitar
#### http://www.cs.ubbcluj.ro/~sanda

In [1]:
class HelperMethods:
    @staticmethod
    def IsDate(text):
#         print("text")
#         print(text)
        for c in text.lstrip():
            if c not in "1234567890 ":
                return False
        return True

In [2]:
import pandas
import requests

In [3]:
page = requests.get('http://www.cs.ubbcluj.ro/~sanda/html/publications/')

In [4]:
data = page.text


In [5]:
from bs4 import BeautifulSoup

In [6]:
soup = BeautifulSoup(data)

In [7]:
pubs = []
for e in soup.find_all('li'):
    if "value" in e.attrs:
#         print(e.contents)
        line = e.contents
        authors = line[0].lstrip('\n ').rstrip('\n ')
        print("authors: ", authors)
        title = line[2]
        print("title: ", title.text)
        affiliation = line[5]
        print("affiliation: ", affiliation.contents[0])
        url = line[5].attrs["href"] if "href" in line[5].attrs else line[5]
        print("url: ", url)
        year = ""
        try:
            year_line = line[8].split()
        
            for i in year_line:
                val = i.split(',')[0]
                if HelperMethods.IsDate(val):
                    year = val
            print("year: ", year)
        except:
            pass
        #year = [k[0] for k in line[8].split(" ") if HelperMethods.IsDate(k[0])][0]
        #print("year: ", year)
        pubs.append((authors, title, affiliation, url, year))
print(len(pubs))

authors:  Sanda (Dragoș) Avram, Diana Șotropa, and Diana Troancă,
title:  Highlighting Trend-Setters in Educational Platforms by Means of Formal Concept Analysis and Answer Set Programming
affiliation:   IFIP Advances in Information and Communication Technology
url:  https://link.springer.com/chapter/10.1007/978-3-319-92928-6_5
year:  2018
authors:  Sanda (Dragoș) Avram, Christian Săcărea, and Diana Șotropa,
title:  Investigating Educational Attractors and Life Tracks in e-Learning Environments Using Formal Concept Analysis
affiliation:  The 25th International Conference on Software, Telecommunications and Computer Networks (SoftCOM)
url:  http://softcom2017.fesb.unist.hr/
year:  2017
authors:  Sanda (Dragoș) Avram, Christian Săcărea, and Diana Troancă,
title:  Using Pattern Structures for Mining Student Behaviour in E-learning Platforms on specific Triggers
affiliation:  The 25th International Conference on Software, Telecommunications and Computer Networks (SoftCOM)
url:  http://soft

In [8]:
for pub in pubs:
    print(pub)

('Sanda (Dragoș) Avram, Diana Șotropa, and Diana Troancă,', <b><i>Highlighting Trend-Setters in Educational Platforms by Means of Formal Concept Analysis and Answer Set Programming</i></b>, <a href="https://link.springer.com/chapter/10.1007/978-3-319-92928-6_5"> IFIP Advances in Information and Communication Technology</a>, 'https://link.springer.com/chapter/10.1007/978-3-319-92928-6_5', '2018')
('Sanda (Dragoș) Avram, Christian Săcărea, and Diana Șotropa,', <b><i>Investigating Educational Attractors and Life Tracks in e-Learning Environments Using Formal Concept Analysis</i></b>, <a href="http://softcom2017.fesb.unist.hr/">The 25th International Conference on Software, Telecommunications and Computer Networks (SoftCOM)</a>, 'http://softcom2017.fesb.unist.hr/', '2017')
('Sanda (Dragoș) Avram, Christian Săcărea, and Diana Troancă,', <b><i>Using Pattern Structures for Mining Student Behaviour in E-learning Platforms on specific Triggers</i></b>, <a href="http://softcom2017.fesb.unist.hr/

### 47 pubs obtained


# DB Storage (TODO)
Time to store the entries in the `papers` DB table.


![Screenshot](Images/PapersTableSpec.PNG)

In [9]:
import mariadb
import json

In [10]:
with open('../credentials.json', 'r') as crd_json_fd:
    json_text = crd_json_fd.read()
    json_obj = json.loads(json_text)

In [11]:
credentials = json_obj["Credentials"]
username = credentials["username"]
password = credentials["password"]

In [13]:
table_name = "publications_cache"
db_name = "ubbcluj"

In [14]:
mariadb_connection = mariadb.connect(user=username, password=password, database=db_name)
mariadb_cursor = mariadb_connection.cursor()

In [16]:
print(table_name)

publications_cache


In [38]:
for paper in pubs:
    
    title = ""
    pub_date = ""
    affiliations = ""
    authors = ""
    print(paper)
    print()
    try:
        pub_date = paper[4].lstrip()
        pub_date = str(pub_date) + "-01-01"
        if len(pub_date) != 10:
            pub_date = ""
    except:
        pass
    
    try:
        authors = paper[0].lstrip()
    except Exception as e:
        print(e)
    
    try:
        affiliations = paper[2].text.lstrip()
    except Exception as e:
        print(e)
    
    try:
#         print(type(paper[1]))
        title = paper[1].text
        if ('\'') in title:
            title = title.split('\'')[0]
    except AttributeError:
        pass
    
    
    
    insert_string = "INSERT INTO {0} SET ".format(table_name) # OK
    insert_string += "Title=\'{0}\', ".format(title) # OK
    insert_string += "ProfessorId=\'{0}\', ".format(6) # OK
    if pub_date != "":
        insert_string += "PublicationDate=\'{0}\', ".format(str(pub_date)) # TODO
    insert_string += "Authors=\'{0}\', ".format(authors) # OK
    insert_string += "Affiliations=\'{0}\' ".format(affiliations) # OK
    print(insert_string) 

    
    try:
        mariadb_cursor.execute(insert_string)
    except mariadb.ProgrammingError as pe:
        print("Error")
        raise pe
    except mariadb.IntegrityError:
        continue

    

('Sanda (Dragoș) Avram, Diana Șotropa, and Diana Troancă,', <b><i>Highlighting Trend-Setters in Educational Platforms by Means of Formal Concept Analysis and Answer Set Programming</i></b>, <a href="https://link.springer.com/chapter/10.1007/978-3-319-92928-6_5"> IFIP Advances in Information and Communication Technology</a>, 'https://link.springer.com/chapter/10.1007/978-3-319-92928-6_5', '2018')

INSERT INTO publications_cache SET Title='Highlighting Trend-Setters in Educational Platforms by Means of Formal Concept Analysis and Answer Set Programming', ProfessorId='6', PublicationDate='2018-01-01', Authors='Sanda (Dragoș) Avram, Diana Șotropa, and Diana Troancă,', Affiliations='IFIP Advances in Information and Communication Technology' 
('Sanda (Dragoș) Avram, Christian Săcărea, and Diana Șotropa,', <b><i>Investigating Educational Attractors and Life Tracks in e-Learning Environments Using Formal Concept Analysis</i></b>, <a href="http://softcom2017.fesb.unist.hr/">The 25th Internation

In [68]:
mariadb_connection.close()

# Conclusion
### In the end, the DB only required ~2 manual modifications (1 deletion, 1 edit) with this code.
45 new insertions.